# Using the Cost class
In this example we will see how to add existing PyBOP cost function to design and fitting problem. Also, the use of custom cost function will be demonstrated.

### Setting up the Environment

Before we begin, we need to ensure that we have all the necessary tools. We will install PyBOP and upgrade dependencies:

In [ ]:
%pip install --upgrade pip ipywidgets -q
%pip install pybop -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Importing Libraries

With the environment set up, we can now import PyBOP alongside other libraries we will need:

In [ ]:
import numpy as np
import pybamm

import pybop

pybop.plot.PlotlyManager().pio.renderers.default = "notebook_connected"

First, to construct a `Cost` class, we need the following objects:
- Model
- Dataset
- Parameters to identify
- Problem

Given the above, we will first construct the model, then the parameters and corresponding dataset. Once that is complete, the problem will be created with a pybop cost assingned to it.

In [ ]:
parameter_set = pybamm.ParameterValues("Chen2020")
model = pybamm.lithium_ion.SPM()

Now that we have the model constructed, let's define the parameters for identification.

In [ ]:
parameters = [
    pybop.Parameter(
        "Negative electrode active material volume fraction",
        initial_value=0.6,
    ),
    pybop.Parameter(
        "Positive electrode active material volume fraction",
        initial_value=0.6,
    ),
]

Next, we generate some synthetic data from the model. This then gets corrupted with Gaussian noise and used to create the Dataset.

In [ ]:
t_eval = np.linspace(0, 10, 100)
sim = pybamm.Simulation(model, parameter_values=parameter_set)
sol = sim.solve(t_eval=t_eval)
voltage = sol["Voltage [V]"](t_eval)
current = sol["Current [A]"](t_eval)
sigma = 0.001
corrupt_voltage = voltage + np.random.normal(0, sigma, len(t_eval))

dataset = pybop.Dataset(
    {
        "Time [s]": t_eval,
        "Current function [A]": current,
        "Voltage [V]": corrupt_voltage,
    }
)

Now that we have the model, parameters, and dataset, we can combine them and construct the problem class with a cost attached.

In [ ]:
builder = (
    pybop.builders.Pybamm()
    .set_dataset(dataset)
    .set_simulation(
        model,
        parameter_values=parameter_set,
    )
    .add_cost(pybop.costs.pybamm.SumOfPower("Voltage [V]"))
)
for param in parameters:
    builder.add_parameter(param)

problem = builder.build()

Now let us see how we can get the cost value for an input.

In [ ]:
problem.set_params([0.5, 0.5])
cost_value = problem.run()
print("Cost:", cost_value)

Cost: [0.08964371]


Often it becomes necessary to create a customised cost function. In the following example, we willsee how to create a custom cost function.

In [ ]:
# Create a custom cost
data = pybamm.DiscreteTimeData(dataset["Time [s]"], dataset["Voltage [V]"], "my_data")
custom_cost = pybop.costs.pybamm.custom(
    "MySumSquaredError",
    pybamm.DiscreteTimeSum((model.variables["Voltage [V]"] - data) ** 2),
    {},
)
builder.add_cost(custom_cost)
problem_custom = builder.build()

Now let us access the created custom cost function.

In [ ]:
problem_custom.set_params([0.5, 0.5])
custom_cost_value = problem_custom.run()
print("Custom cost:", custom_cost_value)

Custom cost: [0.17928742]
